# Tarea 4: Programación Genética

## 4.1 Tabla de verdad


Encontrar las funciones que hacen cierta la tabla de verdad anexa.
Con el propósito de observar el impacto de una buena selección en el conjunto de funciones, se usarán dos F1 = {and, or, not}, F2 = {and, or, not, xor} y el conjunto terminal  T = {A, B, C, [0, 1]}. Donde A, B, C, solo los valores de la tabla adjunta, y 0, 1 son constantes opcionales.

In [1]:
import deap
import math
import operator
import random
import numpy as np
import pandas as pd
from deap import gp, base, creator, tools, algorithms
import plotly.graph_objs as go

In [2]:
# Cargar tabla de verdad desde un archivo CSV
tabla = pd.read_csv('data/Paridad.csv')

In [3]:
tabla

,A,B,C,S
0,0,0,0,1
1,0,0,1,0
2,0,1,0,0
3,0,1,1,1
4,1,0,0,0
5,1,0,1,1
6,1,1,0,1
7,1,1,1,0


Entiendo entonces que hay que ajustar un modelo de programación genética para que aprenda la tabla de verdad dada, usando los dos conjuntos de funciones y el conjunto terminal dado.

In [4]:

# Primitive set con 3 variables A,B,C
pset = gp.PrimitiveSet("MAIN", 3)
pset.renameArguments(ARG0='A')
pset.renameArguments(ARG1='B')
pset.renameArguments(ARG2='C')

# Funciones
pset.addPrimitive(operator.and_, 2)
pset.addPrimitive(operator.or_, 2)
pset.addPrimitive(operator.not_, 1)

# Terminales constantes
pset.addTerminal(0)
pset.addTerminal(1)

# Fitness
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("expr", gp.genFull, pset=pset, min_=1, max_=3)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("compile", gp.compile, pset=pset)

def evalParity(individual):
    func = toolbox.compile(expr=individual)
    # 'tabla' tiene columnas A,B,C,S
    errors = (
        (func(row.A, row.B, row.C) - row.S)**2
        for _, row in tabla.iterrows()
    )
    return math.fsum(errors),

toolbox.register("evaluate", evalParity)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("expr_mut", gp.genFull, min_=0, max_=2)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)

# Limitar altura
toolbox.decorate("mate", gp.staticLimit(key=operator.attrgetter("height"), max_value=5))
toolbox.decorate("mutate", gp.staticLimit(key=operator.attrgetter("height"), max_value=5))

In [5]:
# Ejecutar evolución con F1 (and, or, not)

POP_SIZE = 200
N_GEN = 40
CXPB = 0.8   # probabilidad de cruzamiento
MUTPB = 0.2  # probabilidad de mutación
RANDOM_SEED = 42
random.seed(RANDOM_SEED)

# Estadísticas y registro del mejor
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("min", min)
stats.register("avg", lambda xs: sum(x[0] for x in xs)/len(xs))

hof = tools.HallOfFame(1)

pop = toolbox.population(n=POP_SIZE)

pop, log = algorithms.eaSimple(pop, toolbox, cxpb=CXPB, mutpb=MUTPB, ngen=N_GEN, 
                               stats=stats, halloffame=hof, verbose=True)

best = hof[0]
print("Mejor individuo F1:", best)
print("Altura:", best.height)
print("Fitness (SSE):", best.fitness.values[0])

# Exactitud sobre la tabla (predicción booleana exacta)
func_best = toolbox.compile(expr=best)
aciertos = 0
for _, row in tabla.iterrows():
    pred = func_best(row.A, row.B, row.C)
    aciertos += int(pred == row.S)
accuracy = aciertos / len(tabla)
print(f"Exactitud F1: {accuracy:.3f}")

# Mostrar log en un DataFrame
log_df = pd.DataFrame(log)
log_df.tail()

gen	nevals	min   	avg  
0  	200   	(3.0,)	3.995
1  	172   	(3.0,)	3.965
2  	170   	(3.0,)	3.93 
3  	174   	(3.0,)	3.895
4  	166   	(3.0,)	3.88 
5  	169   	(3.0,)	3.845
6  	164   	(2.0,)	3.72 
7  	165   	(2.0,)	3.635
8  	161   	(2.0,)	3.51 
9  	178   	(2.0,)	3.52 
10 	166   	(2.0,)	3.475
11 	167   	(2.0,)	3.36 
12 	163   	(2.0,)	3.305
13 	177   	(2.0,)	3.145
14 	160   	(2.0,)	3.04 
15 	171   	(2.0,)	2.945
16 	176   	(2.0,)	2.855
17 	186   	(2.0,)	2.795
18 	174   	(2.0,)	2.81 
19 	165   	(2.0,)	2.76 
20 	178   	(2.0,)	2.69 
21 	170   	(2.0,)	2.655
22 	156   	(2.0,)	2.705
23 	168   	(2.0,)	2.675
24 	169   	(2.0,)	2.715
14 	160   	(2.0,)	3.04 
15 	171   	(2.0,)	2.945
16 	176   	(2.0,)	2.855
17 	186   	(2.0,)	2.795
18 	174   	(2.0,)	2.81 
19 	165   	(2.0,)	2.76 
20 	178   	(2.0,)	2.69 
21 	170   	(2.0,)	2.655
22 	156   	(2.0,)	2.705
23 	168   	(2.0,)	2.675
24 	169   	(2.0,)	2.715
25 	177   	(2.0,)	2.595
26 	167   	(2.0,)	2.655
27 	167   	(2.0,)	2.64 
28 	170   	(2.0,)	2.615
29 	178   	(2.0,

,gen,nevals,min,avg
36,36,160,"(2.0,)",2.580
37,37,167,"(2.0,)",2.600
38,38,164,"(2.0,)",2.560
39,39,174,"(2.0,)",2.610
40,40,170,"(2.0,)",2.585


In [6]:
# Repetir con F2 (and, or, not, xor)

# Definir nuevo primitive set con XOR
pset2 = gp.PrimitiveSet("MAIN", 3)
pset2.renameArguments(ARG0='A')
pset2.renameArguments(ARG1='B')
pset2.renameArguments(ARG2='C')
pset2.addPrimitive(operator.and_, 2)
pset2.addPrimitive(operator.or_, 2)
pset2.addPrimitive(operator.not_, 1)
pset2.addPrimitive(operator.xor, 2)  # agregado
pset2.addTerminal(0)
pset2.addTerminal(1)

# Nuevo toolbox (evitar conflicto con el anterior)
creator.create("FitnessMin2", base.Fitness, weights=(-1.0,))
creator.create("Individual2", gp.PrimitiveTree, fitness=creator.FitnessMin2)

toolbox2 = base.Toolbox()
toolbox2.register("expr", gp.genFull, pset=pset2, min_=1, max_=3)
toolbox2.register("individual", tools.initIterate, creator.Individual2, toolbox2.expr)
toolbox2.register("population", tools.initRepeat, list, toolbox2.individual)
toolbox2.register("compile", gp.compile, pset=pset2)

def evalParity2(individual):
    func = toolbox2.compile(expr=individual)
    errors = ((func(r.A, r.B, r.C) - r.S)**2 for _, r in tabla.iterrows())
    return math.fsum(errors),

toolbox2.register("evaluate", evalParity2)
toolbox2.register("select", tools.selTournament, tournsize=3)
toolbox2.register("mate", gp.cxOnePoint)
toolbox2.register("expr_mut", gp.genFull, min_=0, max_=2)
toolbox2.register("mutate", gp.mutUniform, expr=toolbox2.expr_mut, pset=pset2)

toolbox2.decorate("mate", gp.staticLimit(key=operator.attrgetter("height"), max_value=5))
toolbox2.decorate("mutate", gp.staticLimit(key=operator.attrgetter("height"), max_value=5))

POP_SIZE2 = 200
N_GEN2 = 40
CXPB2 = 0.8
MUTPB2 = 0.2
random.seed(123)

stats2 = tools.Statistics(lambda ind: ind.fitness.values)
stats2.register("min", min)
stats2.register("avg", lambda xs: sum(x[0] for x in xs)/len(xs))

hof2 = tools.HallOfFame(1)

pop2 = toolbox2.population(n=POP_SIZE2)

pop2, log2 = algorithms.eaSimple(pop2, toolbox2, cxpb=CXPB2, mutpb=MUTPB2, ngen=N_GEN2,
                                 stats=stats2, halloffame=hof2, verbose=True)

best2 = hof2[0]
print("Mejor individuo F2:", best2)
print("Altura:", best2.height)
print("Fitness (SSE):", best2.fitness.values[0])

func_best2 = toolbox2.compile(expr=best2)
aciertos2 = sum(int(func_best2(r.A, r.B, r.C) == r.S) for _, r in tabla.iterrows())
accuracy2 = aciertos2 / len(tabla)
print(f"Exactitud F2: {accuracy2:.3f}")

import pandas as pd
log2_df = pd.DataFrame(log2)
log2_df.tail()

gen	nevals	min   	avg  
0  	200   	(0.0,)	3.985
1  	175   	(0.0,)	3.965
2  	164   	(0.0,)	3.985
3  	175   	(0.0,)	3.985
4  	174   	(2.0,)	3.945
5  	163   	(0.0,)	3.89 
6  	172   	(0.0,)	3.85 
7  	176   	(0.0,)	3.785
8  	169   	(0.0,)	3.775
9  	169   	(2.0,)	3.75 
10 	175   	(2.0,)	3.715
11 	167   	(0.0,)	3.49 
12 	170   	(0.0,)	3.44 
5  	163   	(0.0,)	3.89 
6  	172   	(0.0,)	3.85 
7  	176   	(0.0,)	3.785
8  	169   	(0.0,)	3.775
9  	169   	(2.0,)	3.75 
10 	175   	(2.0,)	3.715
11 	167   	(0.0,)	3.49 
12 	170   	(0.0,)	3.44 
13 	160   	(0.0,)	3.285
14 	169   	(0.0,)	3.18 
15 	166   	(0.0,)	2.85 
16 	156   	(0.0,)	2.84 
17 	170   	(0.0,)	2.665
13 	160   	(0.0,)	3.285
14 	169   	(0.0,)	3.18 
15 	166   	(0.0,)	2.85 
16 	156   	(0.0,)	2.84 
17 	170   	(0.0,)	2.665
18 	169   	(0.0,)	2.45 
19 	147   	(0.0,)	2.08 
20 	157   	(0.0,)	2.035
21 	164   	(0.0,)	1.955
22 	167   	(0.0,)	2.005
23 	166   	(0.0,)	1.975
24 	165   	(0.0,)	1.945
25 	178   	(0.0,)	1.88 
18 	169   	(0.0,)	2.45 
19 	147   	(0.0,

,gen,nevals,min,avg
36,36,172,"(0.0,)",1.570
37,37,162,"(0.0,)",1.585
38,38,174,"(0.0,)",1.665
39,39,172,"(0.0,)",1.830
40,40,152,"(0.0,)",1.555


## Regresión simbólica con la tabla anexa

Adjunto al presente se incluye la tabla de valores. Para este caso el conjunto de funciones es F = {+, -, *, div, cos, sin, log, exp, abs, sqrt, x^y, ...} sientanse en  libertad de incluir o quitar las funciones según lo crean conveniente. 
A su vez, el conjunto terminal queda definido por  T= {x, y, R, k's}. 
Donde x, y son los valores tomado de la tabla, R el conjunto de reales (recuerden que los pueden generar incluso con aleatorios), y k  son constantes que ustedes consideren que pueden funcionar, (pi, e, ...)


## A

In [7]:
clase = pd.read_csv('data/Reg_Sybol_Class.csv')
clase.columns = ['x', 'f']

In [8]:
clase.head()

,x,f
0,-10,4.85
1,-9,-3.20
2,-8,2.80
3,-7,1.30
4,-6,4.10


In [9]:
# Protected primitive functions

def protectedDiv(a, b):
    return a / b if abs(b) > 1e-9 else a

def protectedLog(a):
    a = float(a)
    return math.log(abs(a)) if abs(a) > 1e-9 else 0.0

def protectedSqrt(a):
    return math.sqrt(abs(a))

def protectedExp(a):
    # Evitar overflow
    a = max(min(a, 50), -50)
    return math.exp(a)

def protectedPow(a, b):
    # Limitar exponentes y manejar dominios
    try:
        if abs(a) > 1e6 or abs(b) > 8:
            return 1.0
        # Forzar exponente entero para estabilidad
        return float(pow(a, int(b)))
    except Exception:
        return 1.0


In [10]:

# ---------- Conjunto de funciones y terminales ----------
pset_sym = gp.PrimitiveSet("MAIN", 1)  # x
pset_sym.renameArguments(ARG0='x')

# Operadores aritméticos
pset_sym.addPrimitive(operator.add, 2)
pset_sym.addPrimitive(operator.sub, 2)
pset_sym.addPrimitive(operator.mul, 2)
pset_sym.addPrimitive(protectedDiv, 2)

# Trig / transcendentales
pset_sym.addPrimitive(math.sin, 1)
pset_sym.addPrimitive(math.cos, 1)
pset_sym.addPrimitive(protectedLog, 1)
pset_sym.addPrimitive(protectedExp, 1)
pset_sym.addPrimitive(abs, 1)
pset_sym.addPrimitive(protectedSqrt, 1)
pset_sym.addPrimitive(protectedPow, 2)

# Constantes
pset_sym.addTerminal(math.pi, name="pi")
pset_sym.addTerminal(math.e, name="e")
pset_sym.addEphemeralConstant("rand", lambda: random.uniform(-5, 5))

# ---------- Clases Fitness / Individuo ----------
try:
    creator.FitnessMinSymb
except AttributeError:
    creator.create("FitnessMinSymb", base.Fitness, weights=(-1.0,))
try:
    creator.IndividualSymb
except AttributeError:
    creator.create("IndividualSymb", gp.PrimitiveTree, fitness=creator.FitnessMinSymb)

# ---------- Toolbox ----------
toolbox_sym = base.Toolbox()
toolbox_sym.register("expr", gp.genHalfAndHalf, pset=pset_sym, min_=1, max_=3)
toolbox_sym.register("individual", tools.initIterate, creator.IndividualSymb, toolbox_sym.expr)
toolbox_sym.register("population", tools.initRepeat, list, toolbox_sym.individual)
toolbox_sym.register("compile", gp.compile, pset=pset_sym)


/Users/roicort/GitHub/PCIC/Genetics/GeneticProgramming/.venv/lib/python3.13/site-packages/deap/gp.py:257: RuntimeWarning: Ephemeral rand function cannot be pickled because its generating function is a lambda function. Use functools.partial instead.
  warnings.warn("Ephemeral {name} function cannot be "


In [11]:
# Función de evaluación para regresión simbólica (ajuste a f(x))
def evalSymbolic(individual):
    func = toolbox_sym.compile(expr=individual)
    try:
        errors = ((func(row['x']) - row['f'])**2 for _, row in clase.iterrows())
        return (math.fsum(errors),)
    except (ValueError, OverflowError, ZeroDivisionError):
        # Penalización alta si ocurre error matemático
        return (1e10,)

toolbox_sym.register("evaluate", evalSymbolic)
toolbox_sym.register("select", tools.selTournament, tournsize=3)
toolbox_sym.register("mate", gp.cxOnePoint)
toolbox_sym.register("expr_mut", gp.genHalfAndHalf, min_=0, max_=2)
toolbox_sym.register("mutate", gp.mutUniform, expr=toolbox_sym.expr_mut, pset=pset_sym)

# Limitar altura de los árboles
toolbox_sym.decorate("mate", gp.staticLimit(key=operator.attrgetter("height"), max_value=6))
toolbox_sym.decorate("mutate", gp.staticLimit(key=operator.attrgetter("height"), max_value=6))

In [12]:
# Ejecutar evolución

POP_SIZE = 200
N_GEN = 40
CXPB = 0.8
MUTPB = 0.2
random.seed(42)

# Estadísticas multi: fitness y tamaño
hof = tools.HallOfFame(1)
stats_fit = tools.Statistics(lambda ind: ind.fitness.values)
stats_size = tools.Statistics(len)
mstats = tools.MultiStatistics(fitness=stats_fit, size=stats_size)
mstats.register("avg", np.mean)
mstats.register("std", np.std)
mstats.register("min", min)
mstats.register("max", max)

pop = toolbox_sym.population(n=POP_SIZE)

pop, log = algorithms.eaSimple(pop, toolbox_sym, cxpb=CXPB, mutpb=MUTPB, ngen=N_GEN,
                               stats=mstats, halloffame=hof, verbose=True)

best = hof[0]
print("Mejor individuo:", best)
print("Altura:", best.height)
print("Error cuadrático (SSE):", best.fitness.values[0])

log_df = pd.DataFrame(log)
log_df.tail()

   	      	                                              fitness                                              	                      size                     
   	      	---------------------------------------------------------------------------------------------------	-----------------------------------------------
gen	nevals	avg        	gen	max                     	min                  	nevals	std        	avg 	gen	max	min	nevals	std    
0  	200   	2.15049e+42	0  	(2.419305433663835e+44,)	(454.89288226747243,)	200   	2.15653e+43	4.42	0  	14 	2  	200   	2.59299
1  	170   	inf        	1  	(inf,)                  	(454.89288226747243,)	170   	nan        	4.2 	1  	18 	1  	170   	2.502  
2  	150   	inf        	2  	(inf,)                  	(454.89288226747243,)	150   	nan        	3.54	2  	13 	1  	150   	2.08528
3  	173   	2.80535e+06	3  	(560918495.3504821,)    	(446.2079654796081,) 	173   	3.95636e+07	3.35	3  	17 	1  	173   	2.35744


/var/folders/4w/72wtbysj7kb189x1nn694p9h0000gn/T/ipykernel_4131/577676481.py:24: RuntimeWarning: divide by zero encountered in scalar power
  return float(pow(a, int(b)))
/Users/roicort/GitHub/PCIC/Genetics/GeneticProgramming/.venv/lib/python3.13/site-packages/numpy/_core/_methods.py:190: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)


4  	175   	9.40841e+41	4  	(1.8816819992712948e+44,)	(431.00214738110503,)	175   	1.32722e+43	3.035	4  	9  	1  	175   	2.16882
5  	174   	8.06435e+41	5  	(1.612870285089681e+44,) 	(431.00214738110503,)	174   	1.13762e+43	2.885	5  	11 	1  	174   	2.18444
6  	179   	9.712e+24  	6  	(1.942400830668118e+27,) 	(428.5607928034756,) 	179   	1.37005e+26	3.15 	6  	11 	1  	179   	2.44489
7  	165   	4.25632e+07	7  	(7846751876.963698,)     	(428.5607928034756,) 	165   	5.54679e+08	3.44 	7  	12 	1  	165   	2.31871
8  	167   	1155.86    	8  	(65402.015157008806,)    	(424.83520185871697,)	167   	4885.52    	3.78 	8  	11 	1  	167   	2.39825
9  	167   	2.77951e+08	9  	(55028809484.68077,)     	(424.83520185871697,)	167   	3.88139e+09	4.085	9  	12 	1  	167   	2.46531
10 	175   	1.34406e+41	10 	(2.688117141816135e+43,) 	(424.83520185871697,)	175   	1.89603e+42	4.415	10 	13 	1  	175   	2.62922
7  	165   	4.25632e+07	7  	(7846751876.963698,)     	(428.5607928034756,) 	165   	5.54679e+08	3.44 	7  	12 	1  

,gen,nevals
36,36,176
37,37,168
38,38,172
39,39,170
40,40,183


In [13]:
# Plot function using plotly

x_vals = clase['x']
y_true = clase['f']
func_best = toolbox_sym.compile(expr=best)
y_pred = [func_best(x) for x in x_vals]

fig = go.Figure()
fig.add_trace(go.Scatter(x=x_vals, y=y_true, mode='lines', name='True', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=x_vals, y=y_pred, mode='lines', name='Predicted', line=dict(color='red')))
fig.show()

## B 

In [14]:
symreg = pd.read_csv('data/symbolic_regression.csv')
symreg.columns = ['x', 'y', 'f']
symreg.head()

,x,y,f
0,-100,-100,19.059929
1,-100,-90,86.073703
2,-100,-80,104.763090
3,-100,-70,78.242133
4,-100,-60,22.679452


In [15]:
# ---------- Conjunto de funciones y terminales ----------

pset_sym2 = gp.PrimitiveSet("MAIN", 2)  # x, y
pset_sym2.renameArguments(ARG0='x')
pset_sym2.renameArguments(ARG1='y')

# Operadores aritméticos
pset_sym2.addPrimitive(operator.add, 2)
pset_sym2.addPrimitive(operator.sub, 2)
pset_sym2.addPrimitive(operator.mul, 2)
pset_sym2.addPrimitive(protectedDiv, 2)
pset_sym2.addPrimitive(protectedPow, 2)
# Trig / transcendentales
pset_sym2.addPrimitive(math.sin, 1)
pset_sym2.addPrimitive(math.cos, 1)
pset_sym2.addPrimitive(protectedLog, 1)
pset_sym2.addPrimitive(protectedExp, 1)
pset_sym2.addPrimitive(abs, 1)
pset_sym2.addPrimitive(protectedSqrt, 1)
# Constantes
pset_sym2.addTerminal(math.pi, name="pi")
pset_sym2.addTerminal(math.e, name="e")
pset_sym2.addEphemeralConstant("rand", lambda: random.uniform(-5, 5))
# ---------- Clases Fitness / Individuo ----------
try:
    creator.FitnessMinSymb2
except AttributeError:
    creator.create("FitnessMinSymb2", base.Fitness, weights=(-1.0,))
try:
    creator.IndividualSymb2
except AttributeError:
    creator.create("IndividualSymb2", gp.PrimitiveTree, fitness=creator.FitnessMinSymb2)
# ---------- Toolbox ----------
toolbox_sym2 = base.Toolbox()
toolbox_sym2.register("expr", gp.genHalfAndHalf, pset=pset_sym2, min_=1, max_=3)
toolbox_sym2.register("individual", tools.initIterate, creator.IndividualSymb2, toolbox_sym2.expr)
toolbox_sym2.register("population", tools.initRepeat, list, toolbox_sym2.individual)
toolbox_sym2.register("compile", gp.compile, pset=pset_sym2)
# Función de evaluación para regresión simbólica (ajuste a f(x,y))
def evalSymbolic2(individual):
    func = toolbox_sym2.compile(expr=individual)
    try:
        errors = ((func(row['x'], row['y']) - row['f'])**2 for _, row in symreg.iterrows())
        return (math.fsum(errors),)
    except (ValueError, OverflowError, ZeroDivisionError):
        # Penalización alta si ocurre error matemático
        return (1e10,)
    
toolbox_sym2.register("evaluate", evalSymbolic2)
toolbox_sym2.register("select", tools.selTournament, tournsize=3)
toolbox_sym2.register("mate", gp.cxOnePoint)
toolbox_sym2.register("expr_mut", gp.genHalfAndHalf, min_=0, max_=2)
toolbox_sym2.register("mutate", gp.mutUniform, expr=toolbox_sym2.expr_mut, pset=pset_sym2)
# Limitar altura de los árboles
toolbox_sym2.decorate("mate", gp.staticLimit(key=operator.attrgetter("height"), max_value=6))
toolbox_sym2.decorate("mutate", gp.staticLimit(key=operator.attrgetter("height"), max_value=6))
# Ejecutar evolución
POP_SIZE = 200
N_GEN = 40
CXPB = 0.8
MUTPB = 0.2
random.seed(42) # Semilla para reproducibilidad

stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("min", min)
stats.register("avg", lambda xs: sum(x[0] for x in xs)/len(xs))
hof = tools.HallOfFame(1)
pop = toolbox_sym2.population(n=POP_SIZE)
pop, log = algorithms.eaSimple(pop, toolbox_sym2, cxpb=CXPB, mutpb=MUTPB, ngen=N_GEN,
                               stats=stats, halloffame=hof, verbose=True)
best = hof[0]
print("Mejor individuo:", best)
print("Altura:", best.height)
print("Error cuadrático (SSE):", best.fitness.values[0])
log_df = pd.DataFrame(log)
log_df.tail()  


gen	nevals	min                  	avg
0  	200   	(2085894.5149631342,)	inf
1  	167   	(2093099.3142058193,)	8.46757e+43
2  	165   	(2075772.5601324833,)	inf        
3  	175   	(2085859.8241450752,)	7.53147e+43
4  	178   	(2072320.1650089845,)	1.92875e+44
5  	173   	(2070226.6959541782,)	3.51221e+44
6  	169   	(2070226.6959541782,)	1.07256e+44
7  	170   	(2070226.6959541782,)	2.29192e+42
8  	167   	(2070226.6959541782,)	1.29726e+44
9  	159   	(2056389.2660578864,)	1.69351e+43
10 	172   	(2056389.2660578864,)	3.51221e+44
11 	170   	(2056389.2660578864,)	2.65373e+28
12 	161   	(2056389.2660578864,)	inf        
13 	167   	(2021670.2285530067,)	inf        
14 	167   	(2021670.2285530067,)	1.64208e+45
15 	166   	(2041854.2418110766,)	1.4207e+44 
16 	167   	(2041854.2418110766,)	5.08054e+43
17 	167   	(2020812.2904222803,)	inf        
18 	172   	(2020812.2904222803,)	5.10477e+43
19 	163   	(2020812.2904222803,)	3.51014e+43
20 	170   	(2020732.0636527701,)	3.31727e+43
21 	181   	(2022137.073511

/Users/roicort/GitHub/PCIC/Genetics/GeneticProgramming/.venv/lib/python3.13/site-packages/deap/gp.py:257: RuntimeWarning:

Ephemeral rand function cannot be pickled because its generating function is a lambda function. Use functools.partial instead.

/var/folders/4w/72wtbysj7kb189x1nn694p9h0000gn/T/ipykernel_4131/577676481.py:24: RuntimeWarning:

divide by zero encountered in scalar power



,gen,nevals,min,avg
36,36,170,"(2001538.6887823662,)",2.066054e+06
37,37,162,"(1997734.7129171528,)",5.510932e+07
38,38,177,"(1997734.7129171528,)",2.083223e+06
39,39,175,"(2001603.084462091,)",2.022429e+09
40,40,168,"(2001603.084462091,)",1.523480e+43


In [16]:
# Ejecutar evolución (multivariable) con estadísticas de fitness y tamaño
POP_SIZE = 200
N_GEN = 40
CXPB = 0.8
MUTPB = 0.2
random.seed(42)

hof = tools.HallOfFame(1)

stats_fit2 = tools.Statistics(lambda ind: ind.fitness.values)
stats_size2 = tools.Statistics(len)
mstats2 = tools.MultiStatistics(fitness=stats_fit2, size=stats_size2)
mstats2.register("avg", np.mean)
mstats2.register("std", np.std)
mstats2.register("min", min)
mstats2.register("max", max)

pop2 = toolbox_sym2.population(n=POP_SIZE)

pop2, log2 = algorithms.eaSimple(pop2, toolbox_sym2, cxpb=CXPB, mutpb=MUTPB, ngen=N_GEN,
                                 stats=mstats2, halloffame=hof, verbose=True)

best2 = hof[0]
print("Mejor individuo multivariable:", best2)
print("Altura:", best2.height)
print("Longitud (nodos):", len(best2))
print("Error cuadrático (SSE):", best2.fitness.values[0])

log2_df = pd.DataFrame(log2)
log2_df.tail()

   	      	                          fitness                          	                      size                     
   	      	-----------------------------------------------------------	-----------------------------------------------
gen	nevals	avg	gen	max   	min                  	nevals	std	avg 	gen	max	min	nevals	std    
0  	200   	inf	0  	(inf,)	(2085894.5149631342,)	200   	nan	4.24	0  	14 	2  	200   	2.46219
1  	167   	8.46757e+43	1  	(6.774055199703722e+45,)	(2093099.3142058193,)	167   	6.27972e+44	3.745	1  	13 	1  	167   	2.19088
2  	165   	inf        	2  	(inf,)                  	(2075772.5601324833,)	165   	nan        	3.745	2  	11 	1  	165   	2.35159
3  	175   	7.53147e+43	3  	(3.65583931381185e+45,) 	(2085859.8241450752,)	175   	4.90913e+44	3.915	3  	11 	1  	175   	2.26446
4  	178   	1.92875e+44	4  	(2.502693694393183e+46,)	(2072320.1650089845,)	178   	1.85437e+45	3.955	4  	15 	1  	178   	2.48656
5  	173   	3.51221e+44	5  	(3.342862250610905e+46,)	(2070226.6959541782,)	17

/var/folders/4w/72wtbysj7kb189x1nn694p9h0000gn/T/ipykernel_4131/577676481.py:24: RuntimeWarning:

divide by zero encountered in scalar power

/Users/roicort/GitHub/PCIC/Genetics/GeneticProgramming/.venv/lib/python3.13/site-packages/numpy/_core/_methods.py:190: RuntimeWarning:

invalid value encountered in subtract



,gen,nevals
36,36,170
37,37,162
38,38,177
39,39,175
40,40,168


In [17]:
# Plot 3D: puntos reales y superficie de la función simbólica encontrada (best2)
# Requiere que best2 y toolbox_sym2 existan (generados en celdas previas)

func_best2 = toolbox_sym2.compile(expr=best2)

# Puntos reales
x_data = symreg['x'].values
y_data = symreg['y'].values
z_true = symreg['f'].values

# Predicciones en los puntos originales (para comparar)
z_pred_pts = [func_best2(xd, yd) for xd, yd in zip(x_data, y_data)]

# Crear una malla para superficie
nx, ny = 40, 40  # resolución de la superficie
x_lin = np.linspace(x_data.min(), x_data.max(), nx)
y_lin = np.linspace(y_data.min(), y_data.max(), ny)
Xg, Yg = np.meshgrid(x_lin, y_lin)
Zg = np.zeros_like(Xg, dtype=float)
for i in range(Xg.shape[0]):
    for j in range(Xg.shape[1]):
        try:
            Zg[i, j] = func_best2(Xg[i, j], Yg[i, j])
        except Exception:
            Zg[i, j] = np.nan

fig3d = go.Figure()

# Superficie del modelo
fig3d.add_trace(go.Surface(x=Xg, y=Yg, z=Zg, colorscale='Viridis', opacity=0.7, name='Modelo'))

# Puntos reales (valor verdadero)
fig3d.add_trace(go.Scatter3d(x=x_data, y=y_data, z=z_true,
                            mode='markers', name='Datos reales',
                            marker=dict(size=4, color='red')))

# (Opcional) Puntos predichos (si se desea comparar directamente)
fig3d.add_trace(go.Scatter3d(x=x_data, y=y_data, z=z_pred_pts,
                            mode='markers', name='Predicción puntos',
                            marker=dict(size=3, color='blue', symbol='circle')))

fig3d.update_layout(title=f'Regresión simbólica: {str(best2)}',
                    scene=dict(xaxis_title='x', yaxis_title='y', zaxis_title='f'),
                    legend=dict(x=0.02, y=0.98))
fig3d.show()

# Métrica simple de ajuste en los puntos originales
sse_pts = float(sum((zp - zt)**2 for zp, zt in zip(z_pred_pts, z_true)))
print('SSE en puntos originales:', sse_pts)


SSE en puntos originales: 1997734.7129171535
